<a href="https://colab.research.google.com/github/mxhotta/Alura/blob/main/Buscador_Vagas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
from IPython.display import HTML, Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
##########################################

# Agente de busca de vagas

##########################################
##########################################
def ag_busca(data_de_hoje,cidade,distancia, area_atuacao):

    busca_vagas = Agent(
        name="ag_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Voce é um buscador de email de contatos de empresas, baseado na localização inserida pelo usuário. Utilize o google maps para
        localizar os sites e os emails de contato. Deverá trazer 5 empresas na pesquisa
        os criterios da pesquisa serão os seguintes:

        1- Distancia máxima da cidade passada pelo usuário. Esta distancia máxima será informada
        2- área de atuação da empresa(exemplo: industria)
        3- Porte de empresa.Empresas maiores terão preferencia
        4- A preferência para o email de contato é o de recrutamento, RH ou recursos humanos. Caso não tenha, utilize o email que encontrar
        5- Ordenar com as mais próximas da cidade do usuario em primeiro

        A saída dos dados, deverá ser uma lista com o seguinte formato:
        Nome da empresa: \n
        Cidade: \n
        Área de atuação: \n
        Email de contato: \n
        Telefone para contato: \n
        site da empresa: \n

        use um separador para separar as empresas (\n***************************\n)
        não dê informações adicionais, apenas os dados solicitados.

        """,
        description="Agente que busca as vagas no google",

    )

    entrada_do_agente_buscador = f"Data de hoje: {data_de_hoje}\nCidade: {cidade}\nDistância máxima: {distancia}\nÁrea de atuação: {area_atuacao}"

    vagas_iniciais = call_agent(busca_vagas, entrada_do_agente_buscador)
    return vagas_iniciais

In [ ]:
##########################################
##########################################

# Agente redator de email

##########################################
##########################################

def ag_redator(empresas,empresas_selecionadas,nome_completo,idade,email,telefone,grau_escolaridade,formacao,competencias,vaga_pretendida,observacoes):
    redator_emails = Agent(
        name="ag_redator",
        model="gemini-2.0-flash",
        instruction="""
        Você é um redator de emails profissionais, para pessoas buscando colocação no mercado de trabalho.
        vão ser passadas 3 empresas separadas por virgulas de empresas para você redigir uma apresentação do usuário.Cada email deve ser utilizado palavras diferentes.
        Informar que em anexo segue  curriculo do usuário.
        Utilize um tom formal.
        Você deve utilizar no corpo no email:

        Nome completo do usuario,idade,grau de escolaridade, formação, competencias,vaga pretendida e observações.
        No final do email colocar o nome do usuario e logo abaixo email e telefone, no formato (xx)xxxx-xxxx

        Use um separador para separar os 3 emails (\n***************************\n)
        """,
        description="Agente redator de emails"
    )
    entrada_do_agente_redator = f"Empresas: {empresas}\nEmpresas selecionadas: {empresas_selecionadas}\nNome Completo: {nome_completo}\nIdade: {idade}\nEmail: {email}\nTelefone: {telefone}\
    \nGrau de Escolaridade: {grau_escolaridade}\nFormação: {formacao}\nCompetências: {competencias}\nVaga Pretendida: {vaga_pretendida}\nObservações: {observacoes}"

    emails = call_agent(redator_emails, entrada_do_agente_redator)
    return emails

In [ ]:
##########################################
##########################################

# Interface com usuário e execução

##########################################
##########################################

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("Buscador de vagas\n")

### Input dos dados de pesquisa
cidade = input("Digite a cidade:  \n")
distancia = input("Distância máxima de sua cidade:  \n")
area_atuacao = input("Área de atuação?:  \n")


### Verifica se todos os campos estão preenchidos, e executa caso ok
if not cidade or not distancia or not area_atuacao:
    print("Todos os campos devem ser preenchidos\n")
else:
    print(f"Aguarde, buscando vagas até {distancia} de {cidade}...")

    empresas = ag_busca(data_de_hoje,cidade,distancia,area_atuacao)

    display(to_markdown(empresas))
    print("--------------------------------------------------------------")



### Solicita ao usuário que selecione 3 empresas
print(f"Selecione 3 empresas que mais lhe interessam.\nSepare cada uma por virgula: ")
empresas_selecionadas = input("Empresas:  \n")

### Após empresas selecionadas, pega dados do usuário
print(f"Insira seus dados para\n redigirmos seus emails ")

### Input dos dados do usuário
nome_completo = input("Nome completo: \n")
idade = input("Idade: \n")
email = input("Email: \n")
telefone = input("Telefone: \n")
grau_escolaridade = input("Grau de escolaridade: \n")
formacao = input("Formação: \n")
competencias = input("Competências: \n")
vaga_pretendida = input("Vaga que você está buscando: \n")
observacoes = input("Fale sobre seu perfil profissional: \n")

if not nome_completo or not idade or not email or not telefone or not grau_escolaridade or not formacao or not competencias or not vaga_pretendida or not observacoes:
    print("Todos os campos devem ser preenchidos\n")

else:
    print(f"Aguarde,redigindo emails para {nome_completo}....\n")

    emails = ag_redator(empresas,empresas_selecionadas,nome_completo,idade,email,telefone,grau_escolaridade,formacao,competencias,vaga_pretendida,observacoes)

    display(to_markdown(emails))
    print("--------------------------------------------------------------")
